# 4. Geocoding and join table with spatial data

In this tutorial, you'll learn about two common manipulations for geospatial data:

- **geocoding**: Find locations(e.g. longitude,latitude) with just the name of a place(e.g. String)
- **table joins**: join data based on spatial relationships


## 4.1 Geocoding

**Geocoding is the process of converting the name of a place or an address to a location on a map**. If you have ever looked up a geographic location based on a landmark description with `Google Maps, Bing Maps, or Baidu Maps`, for instance, then you have used a geocoder!

In this tutorial, We'll use `geopy` to do all of our geocoding.

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings
warnings.filterwarnings('ignore')

 [Nominatim](https://nominatim.openstreetmap.org/) refers to the geocoding software that will be used to generate locations.

We begin by instantiating the geocoder. Then, we need only apply the name or address as a Python string. (In this case, we supply "Pyramid of Khufu", also known as the Great Pyramid of Giza.)

If the geocoding is successful, it returns a `geopy.location.Location` object with two important attributes:

- the `point` attribute contains the (latitude, longitude) location, and
- the `address` attribute contains the full address.

In [2]:
geolocator = Nominatim(user_agent="kaggle_learn")
location = geolocator.geocode("Pyramid of Khufu")

print(location.point)
print(location.address)

29 58m 44.976s N, 31 8m 3.17625s E
هرم خوفو, شارع ابو الهول السياحي, نزلة البطران, الجيزة, 12556, مصر


The value for the `point` attribute is a `geopy.point.Point` object, and we can get the latitude and longitude from the `latitude and longitude` attributes, respectively.

In [3]:
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

Latitude: 29.97916
Longitude: 31.134215625236113


It's often the case that we'll need to geocode many addresses. For instance, say we want to obtain the locations of 100 top universities in Europe.

In [4]:
root_dir="/home/pliu/data_set/kaggle/geospatial/L04"
universities_path=f"{root_dir}/top_universities.csv"

In [5]:
universities = pd.read_csv(universities_path)
universities=universities.head()

Then we can use a lambda function to apply the geocoder to every row in the DataFrame. (We use a try/except statement to account for the case that the geocoding is unsuccessful.)

In [6]:
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = {'init': 'epsg:4326'}
universities.head()

100.0% of addresses were geocoded!


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.752546,-1.214330,POINT (-1.21433 51.75255)
1,University of Cambridge,52.200623,0.110474,POINT (0.11047 52.20062)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.408327,8.507564,POINT (8.50756 47.40833)
4,UCL,51.521764,-0.135120,POINT (-0.13512 51.52176)


Next, we visualize all of the locations that were returned by the geocoder. Notice that a few of the locations are certainly inaccurate, as they're not in Europe!

In [12]:
# Create a map
universities_map = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=3)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(universities_map)

# Display the map
universities_map

## 4.2 Table joins

### 4.2.1 Attribute join

You already know how to use `pd.DataFrame.join()` to combine information from multiple DataFrames with a shared index. We refer to this way of joining data (by simpling matching values in the index) as an `attribute join`.

When performing an attribute join with a GeoDataFrame, it's best to use the `gpd.GeoDataFrame.merge()`. To illustrate this, we'll work with a `GeoDataFrame` europe_boundaries containing the boundaries for every country in Europe. The first five rows of this GeoDataFrame are printed below.


In [7]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [8]:
europe_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((178.72530 71.09880, 180.00000 ..."
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3..."
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."


We'll join it with a DataFrame europe_stats containing the estimated population and gross domestic product (GDP) for each country.

In [9]:
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,144373535.0,1699876
1,Norway,5347896.0,403336
2,France,67059887.0,2715518
3,Sweden,10285453.0,530883
4,Belarus,9466856.0,63080


We do the attribute join in the code cell below. The **on** argument is set to the column name that is used to match rows in `europe_boundaries` to rows in `europe_stats`.

In [10]:
# Use an attribute join to merge data about countries in Europe
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((178.72530 71.09880, 180.00000 ...",144373535.0,1699876
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",5347896.0,403336
2,France,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3...",67059887.0,2715518
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",10285453.0,530883
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",9466856.0,63080


## 4.2.2 Spatial join
Another type of join is a **spatial join**. With a spatial join, we combine GeoDataFrames based on the `spatial relationship` between the objects in the "geometry" columns. For instance, we already have a GeoDataFrame universities containing geocoded addresses of European universities.

Then we can use a spatial join to match each university to its corresponding country. We do this with `gpd.sjoin()`.

> Note the sjoin() method requires a python dependency rtree. Use below instruction to install it

```shell
sudo apt install libspatialindex-dev
poetry add rtree
```

In [11]:
# Use spatial join to match universities to countries in Europe
european_universities = gpd.sjoin(universities, europe)

# Investigate the result
print("We located {} universities.".format(len(universities)))
print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))

european_universities.head()

We located 5 universities.
Only 5 of the universities were located in Europe (in 2 different countries).


,Name,Latitude,Longitude,geometry,index_right,name,pop_est,gdp_md_est
0,University of Oxford,51.752546,-1.214330,POINT (-1.21433 51.75255),28,United Kingdom,66834405.0,2829108
1,University of Cambridge,52.200623,0.110474,POINT (0.11047 52.20062),28,United Kingdom,66834405.0,2829108
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896),28,United Kingdom,66834405.0,2829108
4,UCL,51.521764,-0.135120,POINT (-0.13512 51.52176),28,United Kingdom,66834405.0,2829108
3,ETH Zurich,47.408327,8.507564,POINT (8.50756 47.40833),19,Switzerland,8574832.0,703082


The spatial join above looks at the `geometry` columns in both `GeoDataFrames`. If a `Point object` from the `universities` GeoDataFrame intersects a `Polygon object` from the `europe` DataFrame, the corresponding rows are combined and added as a single row of the `european_universities` DataFrame. Otherwise, countries without a matching university (and universities without a matching country) are omitted from the results.

The **gpd.sjoin()** method is customizable for different types of joins, through the how and `op` arguments. For instance, you can do the equivalent of a SQL `left (or right)` join by setting `how='left'` (or how='right'). We won't go into the details in this course, but you can learn more in the [documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html).